In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00


In [2]:
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd
import torch

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Data processing

In [8]:
train_data = pd.read_csv('/content/drive/MyDrive/224ndata/train_dataset.csv')
question = "Evaluate the completeness of the given story. Answer only with a proportion, in the form of a decimal. Do not include any other details. Story: "
dataset = Dataset.from_pandas(train_data)


def transform_story(example):
    story_text = example['story']
    error_position = example['chunk_prop']

    formatted_text = f'[INST] Evaluate the completeness of the given story. Answer only with a proportion, in the form of a decimal. Do not include any other details. Story: {story_text} [/INST] {error_position} '

    return {'text': formatted_text}

transformed_dataset = dataset.map(transform_story)
for i in range(5):
    print(transformed_dataset[i]['text'])

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

[INST] Evaluate the completeness of the given story. Answer only with a proportion, in the form of a decimal. Do not include any other details. Story: It was a damp and dreary day in London when a telegram arrived at 221B Baker Street, summoning Sherlock Holmes and Dr. John Watson to a case that would test the limits of their deduction skills. The year was 1892, and the mystery surrounding the murder of Scott Eccles' host, Garcia, also known as the Tiger of San Pedro, was shrouded in darkness and intrigue.

With a sense of anticipation, Holmes and Watson made their way to the scene of the crime, a grand house in the outskirts of London where the gruesome murder had taken place. Miss Burnet, the governess of the household, met them with a look of fear and determination in her eyes. She revealed that she was part of a secret society seeking revenge on Garcia for his atrocities in San Pedro, a fact that sent shivers down the spines of both Holmes and Watson.

As they delved deeper into th

In [9]:

model_name = "meta-llama/Llama-2-7b-chat-hf"

### QLoRA parameters

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

### bitsandbytes parameters

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

### TrainingArguments parameters

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

### SFT parameters

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [10]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("Llama-2-7b-unresolved-error-finetuned")

model.eval()

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.361100
50,1.119300
75,1.056300
100,1.039100
125,1.012100
150,1.031700
175,1.019000
200,1.023100


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): 

In [11]:
test_df = pd.read_csv('/content/drive/MyDrive/224ndata/test_dataset.csv').reset_index(drop=True).dropna()
def remove_empty_lines(text):
  lines = text.split('\n')
  non_empty_lines = [line.strip() for line in lines if line.strip()]
  return '\n'.join(non_empty_lines)

# Apply the function to the text column
test_df['story'] = test_df['story'].apply(remove_empty_lines)
test_df['story_truncated'] = test_df['story_truncated'].apply(remove_empty_lines)

In [ ]:
import re
from tqdm import tqdm
y_true = []
y_pred = []
responses = []

for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    story = row['story_truncated']
    proportion = row['chunk_prop']
    prompt = f"[INST] Estimate how complete the following story is, in terms of plot development. Answer only as a proportion. Do not include any other detaisl. Story: {story} [/INST]"
    # Generate model output

    model_inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    output = model.generate(**model_inputs,  max_length=1024)
    print(output)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    print(response)
    responses.append(response)


    y_true.append(proportion)

  1%|          | 1/101 [00:31<52:01, 31.22s/it]

tensor([[    1,   518, 25580,  ..., 29946, 29955, 29900]], device='cuda:0')
[INST] Estimate how complete the following story is, in terms of plot development. Answer only as a proportion. Do not include any other detaisl. Story: In the secluded mountains behind Virginia, nestled in the mist-shrouded valleys where the trees whispered ancient secrets and the wind carried a haunting melody, there stood the stone house of Doomdorf. A man of dark repute, Doomdorf was known for his illicit liquor that held a devilish allure to those who dared to taste it. The locals whispered tales of his shady past and the chaos his hellish brew brought upon the community.
Uncle Abner, a wise and respected elder of the area, and Squire Randolph, a man of authority and influence, grew concerned about the impacts of Doomdorf's activities on the community. Together, they decided to confront the mysterious man and put an end to his nefarious deeds once and for all.
One fateful evening, they made their way to Do

  2%|▏         | 2/101 [01:35<1:23:47, 50.78s/it]

tensor([[    1,   518, 25580,  ..., 29955, 29941, 29953]], device='cuda:0')
[INST] Estimate how complete the following story is, in terms of plot development. Answer only as a proportion. Do not include any other detaisl. Story: Frances Wilton had been battling with her addiction to morphia for years, sinking deeper and deeper into its clutches until she reached her lowest point. Desperate to break free from the chains of her dependence, she sought out the help of Dr. Halifax, a renowned physician known for his unorthodox but effective methods of treating addiction.
Dr. Halifax took on Frances' case with determination and compassion, knowing the uphill battle she faced in overcoming her addiction. He devised a clever treatment plan that involved substituting morphia with strychnine, a potent tonic that mimicked some of the effects of the drug but without the addictive properties. [/INST] 0.47368421052631576 0.47368421052631576 0.47368421052631576 0.47368421052631576 0.47368421052631576

  3%|▎         | 3/101 [01:43<51:15, 31.39s/it]  

...

tensor([[    1,   518, 25580,  ..., 29947, 29945, 29955]], device='cuda:0')
[INST] Estimate how complete the following story is, in terms of plot development. Answer only as a proportion. Do not include any other detaisl. Story: Professor Kennedy sat at his desk, engrossed in a book on forensic psychology, when his assistant, Walter, brought in the day's mail. Among the usual assortment of letters and flyers, there was one that caught Kennedy's attention. It was a plea for help from a woman named Janet Cranston, who claimed to be held against her will in a private asylum called Belleclaire Sanatorium.
Kennedy's curiosity was piqued. He read the letter carefully, noting the desperation in the woman's words and the urgency of her situation. Without hesitation, he decided to take up the case and investigate the mysterious goings-on at Belleclaire Sanatorium.
Walter was skeptical at first, but he trusted Kennedy's instincts and agreed to assist him in the daring rescue mission. They made p

 19%|█▉        | 19/101 [12:54<1:02:09, 45.48s/it]

tensor([[    1,   518, 25580,  ..., 29906, 29941, 29947]], device='cuda:0')
[INST] Estimate how complete the following story is, in terms of plot development. Answer only as a proportion. Do not include any other detaisl. Story: The Hon. Staunton Toleron was a man of impeccable taste and connections, always one step ahead in the intricate dance of high society. So when he managed to secure a luxurious suite at the prestigious Hotel Cyril, it raised no eyebrows. His arrival was met with polite nods and deferential smiles, as befitting a man of his stature.
But as the sun dipped below the horizon and the city lights sparkled outside his window, the tranquility of the hotel was shattered by a frantic commotion. The famous actress Betty Barry, staying in the suite next to Toleron's, had discovered her precious jewels missing. Accusations flew, fingers pointed, and suspicion fell heavily on the Hon. Staunton Toleron.
Toleron's protests of innocence fell on deaf ears as the evidence seemed t

 20%|█▉        | 20/101 [13:42<1:02:35, 46.36s/it]

tensor([[    1,   518, 25580,  ..., 29871, 29900, 29889]], device='cuda:0')
[INST] Estimate how complete the following story is, in terms of plot development. Answer only as a proportion. Do not include any other detaisl. Story: Miss Brooke, a renowned detective with a sharp mind and a keen eye for detail, was enjoying a quiet afternoon at her cozy home when a knock on the door disrupted her peace. Inspector Clampe stood on her doorstep, his face a mix of urgency and intrigue.
"Miss Brooke, I need your assistance with a rather peculiar case that has come to our attention," Clampe explained, his voice tinged with anticipation.
Intrigued, Miss Brooke invited the inspector in and listened intently as he unraveled the details of the mystery. It all began with the theft of a valuable cheque from a local business owner. The unusual aspect of the case was a reported ghost sighting at the scene of the crime, adding a layer of intrigue to the already baffling situation.
As Miss Brooke delved de